This notebook is used after the inventory is created. It is creating config files for each recording and creating nwb files for spike times and psth (if avail). Finally validating the nwb files, it then uploades the files to dandi. 

In [1]:
from datetime import datetime
from uuid import uuid4
import numpy as np
import scipy.io
import os, yaml, glob, json
import pandas as pd
from nwbwidgets import nwb2widget
from pynwb import NWBHDF5IO, NWBFile
from pynwb.file import Subject
import shutil
import logging
import h5py
import pytz
from tqdm import tqdm
from utils.nwb_helper import  create_nwb


In [8]:
############### Create Custom Config Files for Each Recording #################
###############################################################################
from utils.config_helper import create_yaml
import os
dir = '/braintree/home/aliya277/inventory/'
array_meta_path  = '/braintree/data2/active/users/sgouldin/array-metadata'
for files in os.listdir(dir):
    num_files = 0
    path = os.path.join(dir, files)
    try: 
        num_files = len(os.listdir(os.path.join(path,'SpikeTimes')))
    except: pass
    
    if num_files == 192: 
        array_metadata = os.path.join(array_meta_path, '021023_pico_mapping_noCIT_adapter_version.json')
        adapter_info_avail = True
    elif num_files == 288: 
        array_metadata = os.path.join(array_meta_path,'pico_firstmapping_Lhem_2023.json')
        adapter_info_avail = False
        
    create_yaml(path, array_metadata, adapter_info_avail)

In [2]:
from dateutil.tz import tzlocal
from datetime import datetime
from uuid import uuid4
import yaml
import os
import json
import numpy as np
from ruamel.yaml import YAML
from utils.rhd_helper import read_header
import configparser


def load_array_metadata(array_path):
    array          = open(array_path)
    data           = json.load(array)
    
    bank_assignment     = np.unique(np.array(list(data['bank'].values())))              # unique for each group
    indices             = []
    for bank in bank_assignment:
        indices.append(np.where(np.array(list(data['bank'].values()))==bank)[0][0])

    num_arrays          = len(bank_assignment)
    subregions          = np.array(list(data['subregion'].values()))[indices]         # unique for each group
    region              = np.array(list(data['region'].values()))[indices]            # same for all
    hemispheres         = np.array(list(data['hemisphere'].values()))[indices]        # same for all
    serialnumbers       = np.array(list(data['arr'].values()))[indices]               # same for all

    positions           = np.zeros((len(list(data['subregion'].values())), 3))          # row, col, number
    positions[:,0]      = np.array(list(data['row'].values()))
    positions[:,1]      = np.array(list(data['col'].values()))
    positions[:,2]      = np.array(list(data['elec'].values()))

    if num_arrays ==6:
        adapter_versions    = np.array(list(data['adapter_version'].values()))[indices]
        return subregions, hemispheres, region, serialnumbers, bank_assignment, positions, num_arrays, adapter_versions
    
    return subregions, hemispheres, region, serialnumbers, bank_assignment, positions, num_arrays
       
def load_rec_info(path):
    with open(os.path.join(path,"RecInfo.yaml") , "r") as f:
        return yaml.load(f, Loader = yaml.FullLoader)

def load_intan_info(Rec_Info):
    intan_info      = Rec_Info['intanproc'].split('/')[:-1]
    intan_info[-2]  = 'intanraw'
    intan_info_path = os.path.join('/', *intan_info, 'info.rhd')
    fid = open(intan_info_path, 'rb')
    return read_header(fid)


In [71]:
def load_excel_sarah():
    df_sarah = pd.read_excel( '/braintree/home/aliya277/dandi_brainscore/Pipeline Monkey Schedule New.xlsx' ,sheet_name='pico' )
    new_header = df_sarah.iloc[0]  
    df_sarah = df_sarah[1:]       
    df_sarah.columns = new_header  
    df_sarah = df_sarah.fillna('empty') 
    return df_sarah

def create_yaml(storage_dir, df, array_metadata_path, adapter_info_avail=False):

    date = f"20{df['date']}"
    if len(str(df['time'])) != 6: time = f"0{df['time']}"
    else: time = str(df['time'])
    
    if df['ImageSet'] == 'normalizers':
        directory = f'norm_FOSS.sub_pico.{date}_{time}.proc'
    elif df['ImageSet'] == 'normalizers-HVM':
        directory = f'norm_HVM.sub_pico.{date}_{time}.proc'
    else: 
        directory = f"exp_{df['ImageSet']}.sub_pico.{date}_{time}.proc"

    imagesetdir = os.path.join(storage_dir, ".".join(directory.split(".")[0:1]))
    subjectdir  = os.path.join(storage_dir, imagesetdir, ".".join(directory.split(".")[0:2]))

    config_dict = dict()

    config_dict['subject'] = {
                'subject_id':   'pico',
                'date_of_birth': datetime(2014, 6, 22, tzinfo = tzlocal()), 
                'sex':          'M',
                'species':      'Macaca mulatta',
                'description':  'monkey'
                }
    
    config_dict['general'] ={
                'experiment_info': {
                    'experiment_description': f'Task: Rapid serial visual presentation (RSVP).',
                    'keywords': ['Vistual Stimuli', 'Object Recognition', 'Inferior temporal cortex (IT)', 'Ventral visual pathway'],
                    'surgery' : '3x Utah Array Implant + Headpost',
                    },
                'lab_info': {
                    'university':  'Massachusetts Institute of Technology',
                    'institution': 'McGovern Institute for Brain Research',
                    'lab': 'DiCarlo',
                    'experimenter': 'Goulding, Sarah',
                    }
                }

    config_dict['hardware'] = {
                    'electrode_name': 'Electrode',
                    'electrode_description': 'Utah CerePort Array with a single electrode array',
                    'electrode_manuf': '2020 Blackrock Microsystems, LLC',
                    'system_name': 'RecordingSystem',
                    'system_description': 'RHD Recording System',
                    'system_manuf': '2010-2023 Intan Technologies', 
                    'adapter_name': 'Utah Array Pedestal Connector',
                    'adapter_description': 'Connects Utah Pedestal to Intan RHD Recording System',
                    'adapter_manuf': 'Ripple Neuro', 
                    'photodiode_name': 'DET36A2 Biased Si Detector',
                    'photodiode_description': 'Photodiode for detecting image presentation times. Comes with DET2A Power Adapter',
                    'photodiode_manuf': 'Thorlabs Inc.', 
                    'monitor_name': 'LG UltraGear',
                    'monitor_description': 'LG 32GP850-B 32 UltraGear QHD (2560 x 1440) Nano IPS Gaming Monitor w/ 1ms (GtG) Response Time & 165Hz Refresh Rate.\
                        Manually color calibrated and set to 120 HZ refresh rate.',
                    'monitor_manuf': 'LG', 
                    }
    
    config_dict['software'] ={
                    'mwclient_version': 'Version 0.11 (2022.02.15)',
                    'mwserver_version': 'Version 0.11 (2022.02.15)',
                    'OS': 'macOS Monterery on MAC Pro (Late 2013)',
                    'intan_version': 'Version 3.1.0'
                    }
   
                
    config_dict['metadata'] = {
                    'nwb_version' : '2.6.0',
                    'file_create_date': datetime.strptime(date, "%Y%m%d"),
                    'identifier': str(uuid4()), 
                    'session_start_time':datetime.strptime(date+time, "%Y%m%d%H%M%S")
                    }
    
    
    config_dict['paths'] = {
                    'SpikeTime': df['Path: SpikeTimes'],
                    'psth': df['Path: h5'],
                    }

    df_sarah = load_excel_sarah()
    text = 'Recording Information'
    try:
        rec_info = df_sarah.iloc[int(df['(excel) Index'])-3][2:14]
        text += str(rec_info)

    except: pass

    config_dict["session_info"] = {
                'session_id': directory,
                'session_description': text,
                }
    

    config_dict['PSTH info'] = {}

    psth_info = configparser.ConfigParser()
    psth_info.read('/braintree/data2/active/users/sgouldin/spike-tools-chong/spike_tools/spike_config.ini')
    for option in psth_info.options('PSTH'):
        config_dict['PSTH info'][option] = psth_info.get('PSTH', option)

    config_dict['Filtering info'] = {}

    for option in psth_info.options('Filtering'):
        config_dict['Filtering info'][option] = psth_info.get('Filtering', option)
                
    
    if adapter_info_avail == True:
        subregions, hemispheres, region, serialnumbers, bank_assignment, positions, num_arrays, adapter_versions = load_array_metadata(array_metadata_path)
    else:
        subregions, hemispheres, region, serialnumbers, bank_assignment, positions, num_arrays = load_array_metadata(array_metadata_path)
    
    config_dict['array_info'] = {'intan_electrode_labeling_[row,col,id]':json.dumps(positions.tolist())}

    if num_arrays == 6:
        for arr, i in zip(bank_assignment, range(num_arrays)) :
                config_dict['array_info']['array_{}'.format(arr)] = {
                            'position': [0.0,0.0,0.0],
                            'serialnumber':     str(serialnumbers[i]),
                            'adapterversion':   str(adapter_versions[i]),
                            'hemisphere':       str(hemispheres[i]),
                            'region':           str(region[i]),
                            'subregion':        str(subregions[i]),
                            }  
    else:
         for arr, i in zip(bank_assignment, range(num_arrays)) :
                config_dict['array_info']['array_{}'.format(arr)] = {
                            'position': [0.0,0.0,0.0],
                            'serialnumber':     str(serialnumbers[i]),
                            'hemisphere':       str(hemispheres[i]),
                            'region':           str(region[i]),
                            'subregion':        str(subregions[i]),
                            }  
    print(directory)

    yaml = YAML()
    with open(os.path.join(subjectdir,directory,f"config_nwb.yaml"), 'w') as yamlfile:
        yaml.dump((config_dict), yamlfile)


In [73]:
df = pd.read_excel( '/braintree/home/aliya277/dandi_brainscore/pico_inventory.xlsx'  )
SubjectName = 'pico'
storage_dir = '/braintree/home/aliya277/inventory'
array_meta_path  = '/braintree/data2/active/users/sgouldin/array-metadata'

# for imageset, excelname, date_, time_, hasspike, hash5, spikepath, h5path in zip(df['ImageSet'], df['(excel) Stimuli'], df['date'], df['time'], df['Has SpikeTime'], \
    # df['Has h5'], df['Path: SpikeTimes'], df['Path: h5']):
    # date = f'20{DataFrame['date']}'
    # if len(str(DataFrame['time'])) != 6:
        # time = f'0{DataFrame['time']}'

for index, DataFrame in df.iterrows():
        
    if DataFrame['Has SpikeTime'] == 1:

        num_files = len(os.listdir(DataFrame['Path: SpikeTimes']))
        if num_files == 192: 
            array_metadata = os.path.join(array_meta_path, '021023_pico_mapping_noCIT_adapter_version.json')
            adapter_info_avail = True
        elif num_files == 288: 
            array_metadata = os.path.join(array_meta_path,'pico_firstmapping_Lhem_2023.json')
            adapter_info_avail = False

        create_yaml(storage_dir, DataFrame, array_metadata, adapter_info_avail)


norm_FOSS.sub_pico.20220616_111545.proc
norm_FOSS.sub_pico.20220615_113442.proc
norm_FOSS.sub_pico.20220907_142157.proc
norm_FOSS.sub_pico.20220929_170635.proc
norm_FOSS.sub_pico.20230216_145353.proc
norm_FOSS.sub_pico.20230221_124922.proc
norm_FOSS.sub_pico.20230224_142014.proc
norm_FOSS.sub_pico.20230303_152601.proc
norm_FOSS.sub_pico.20230328_145456.proc
norm_FOSS.sub_pico.20230428_111937.proc
norm_FOSS.sub_pico.20230508_120630.proc
norm_FOSS.sub_pico.20230509_112900.proc
norm_FOSS.sub_pico.20230510_125550.proc
norm_FOSS.sub_pico.20230511_120927.proc
norm_FOSS.sub_pico.20230515_133554.proc
norm_FOSS.sub_pico.20230516_120137.proc
norm_FOSS.sub_pico.20230605_122450.proc
norm_FOSS.sub_pico.20230606_134244.proc
norm_FOSS.sub_pico.20230609_133109.proc
norm_FOSS.sub_pico.20230613_124343.proc
norm_FOSS.sub_pico.20230616_100142.proc
norm_FOSS.sub_pico.20230621_140747.proc
norm_FOSS.sub_pico.20230630_135832.proc
norm_FOSS.sub_pico.20230705_122755.proc
norm_FOSS.sub_pico.20230707_140510.proc


FileNotFoundError: [Errno 2] No such file or directory: '/braintree/home/aliya277/inventory/norm_FOSS/norm_FOSS.sub_pico/norm_FOSS.sub_pico.20230912_103737.proc/config_nwb.yaml'

Date: empty
Stimuli: facesMSFDE
Fixation Window: '2'
Visual Deg: 8
ON/OFF (ms): 100/100
Reps: 30
Stimuli per Trial: 8
Images*/Videos: 360
Total images/Videos: 10800
Reward (ms): 65-70
Total Water (ml): empty
aIT Pedestal: empty
pIT Pedestal: empty
Referencing: empty
Notes: empty
2021-12-01 00:00:00: empty
.nan: empty
intanproc: 
  /braintree/data2/active/users/sgouldin/projects/facesMSFDE/monkeys/pico/intanproc/ko-pico-facesMSFDE_230327_134955/spikeTime
SessionDate: '20230327'
SessionTime: '134955'


In [18]:
############### Iterate through every File and Create NWB #####################
###############################################################################

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
i = 0
print(f'Number of all files: {len(all_files)}')
for folder in all_files[0:1]:
    print(f'Creating nwb for folder {folder}')
    path = os.path.join(inventory, folder)
    SessionName = folder
    # try:
    with open(os.path.join(path,"config_nwb.yaml") , "r") as f:
        config = yaml.load(f, Loader = yaml.FullLoader)

    nwbfile = create_nwb(config, path)

    # if os.path.isfile(os.path.join(path, f"{SessionName}.nwb")):
    #     os.remove(os.path.join(path, f"{SessionName}.nwb"))

    print('Saving NWB File..')
    io = NWBHDF5IO(os.path.join(path, f"{SessionName}.nwb"), "w") 
    io.write(nwbfile)
    io.close()
    print(f"File {i} saved.")
    i += 1
    # except: print(f'This file did not work: {SessionName}')

Number of all files: 201
Creating nwb for folder exp_Alireza_paradigm1.sub_pico.20230616_103724.proc
Saving NWB File..
File 0 saved.


In [ ]:
############### ADDITIONAL BOX ################################################
###############################################################################
"""
Adding psth meta to nwb. file. This is now added to the creation process - no need to run this box again.
"""

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
i = 0
print(f'Number of all files: {len(all_files)}')
for folder in all_files[0:1]:
    print(f'Adding psth meta to nwb: {folder}')
    path = os.path.join(inventory, folder)
    SessionName = folder
    filename = os.path.join(path, f"{SessionName}.nwb")

    if 'h5Files' in os.listdir(path):

        filename_h5 = os.listdir(os.path.join(path, 'h5Files'))[0]
        file = h5py.File(os.path.join(path, 'h5Files', filename_h5),'r+') 
        meta = np.array([np.array(file['meta']['start_time_ms']), np.array(file['meta']['stop_time_ms']), np.array(file['meta']['tb_ms'])])
        file.close()

        with NWBHDF5IO(filename, "r+") as io:
            read_nwbfile = io.read()

            try:
                read_nwbfile.add_scratch(
                    meta,
                    name="psth meta",
                    description="start_time_ms, stop_time_ms, tb_ms",
                    )
                
                io.write(read_nwbfile)
            except: pass # meta already exists

            io.close()
        
    

In [10]:
############### Check if All Files are Written and can be Opened ##############
###############################################################################

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
i = 0
for folder in all_files:
        path = os.path.join(inventory, folder)
        try:
                io = NWBHDF5IO(os.path.join(path, f"{folder}.nwb"), "r") 
                nwbfile = io.read()
                # display(nwbfile)
                io.close()
        except: print(f'{i}: This File can not be opened: {folder}')
        i += 1


In [24]:
############### Validate All Files Using pwnyb, nwbinspectors #################
###############################################################################

from pynwb import validate
from nwbinspector import inspect_nwbfile
from dandi.validate import validate as dandival

inventory   = '/braintree/home/aliya277/inventory/'
all_nwb_paths = []
all_files = os.listdir(inventory)
for folder in all_files:
    path = os.path.join(inventory, folder, f'{folder}.nwb')
    all_nwb_paths.append(path)
    
pynwb_validation = validate(paths = all_nwb_paths)
dandi_validation = dandival(all_files)

nwbinspector_validation = []
for path in all_nwb_paths:
    results = list(inspect_nwbfile(nwbfile_path=path))
    nwbinspector_validation.append(results)

In [ ]:
pynwb_validation

In [ ]:
nwbinspector_validation

# extra

In [ ]:
############### Add PSTH if Needed ############################################
###############################################################################

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
for folder in all_files:
        path = os.path.join(inventory, folder)
        try:
                io = NWBHDF5IO(os.path.join(path, f"{folder}.nwb"), "r") 
                nwbfile = io.read()
                io.close()
        except: print(f'This File can not be opened: {folder}')

In [70]:
nwb2widget(nwbfile)

In [ ]:
# if 'h5Files' in os.listdir(path):
#     print('Opening PSTH...')
#     filename = os.listdir(os.path.join(path, 'h5Files'))[0]
#     file = h5py.File(os.path.join(path, 'h5Files', filename),'r+') 
#     data = file['psth'][:]
#     file.close()


#     print('Adding PSTH...')
#     nwbfile.add_scratch(
#         data,
#         name="psth",
#         description="psth, uncorrected [channels x stimuli x reps x timebins]",
#         )
    
# else: counter +=1

# try: io.close()
# except:pass

# if os.path.isfile(os.path.join(path, f"{SessionName}.nwb")):
#         os.remove(os.path.join(path, f"{SessionName}.nwb"))

# print('Saving NWB File..')
# io = NWBHDF5IO(os.path.join(path, f"{SessionName}.nwb"), "w") 
# io.write(nwbfile)
# io.close()
# print("File saved.")

#     # psth    = get_psth_from_nwb(nwbfile, path, start_time, stop_time, timebin)
#     # data    = psth['psth']

# print(f'{counter} SpikeTimes do not have an h5 file.')

In [43]:
inventory   = '/braintree/home/aliya277/inventory/'

def find_directories_without_extension(root_dir, extension):
    directory_paths = []
    for foldername, subfolders, filenames in os.walk(root_dir):
        depth = foldername[len(root_dir):].count(os.sep)
        if depth ==  0:
            # Check if any file in the directory has the specified extension
            if not any(filename.endswith(extension) for filename in filenames):
                directory_paths.append(os.path.join(root_dir, foldername))
                # print(os.path.join(root_dir, foldername))
    return directory_paths[1:]

print(len(find_directories_without_extension(inventory, '.nwb')))


45


In [2]:
import gc
for obj in gc.get_objects():   # Browse through ALL objects
    if isinstance(obj, h5py.File):   # Just HDF5 files
        try:
            obj.close()
        except: pass

In [5]:
def read_names(filename):
    assignment  = filename.split('.')[0].split('-')[1]
    number      = filename.split('.')[0].split('-')[2]
    return np.asarray([assignment, number])

def create_nwb(config, path):

    SessionDate = path.split('/')[-1].split('.')[-2].split('_')[0]
    SessionTime = path.split('/')[-1].split('.')[-2].split('_')[1]
    SubjectName = path.split('/')[-1].split('.')[1].split('_')[1]
    date_format = "%Y%m%d %H%M%S"

    if config['subject']['subject_id'].lower() != SubjectName.lower():
        raise ValueError("Subject Name incorrect.")
        
    session_start_time_ = datetime.strptime(SessionDate+' '+SessionTime, date_format)
    # Define the timezone you want to use (e.g., 'US/Eastern' for Boston)
    desired_timezone = pytz.timezone('US/Eastern')
    session_start_time = desired_timezone.localize(session_start_time_)

    ################ CREATE NWB FILE WITH METADATA ################################
    ###############################################################################
    nwbfile = NWBFile(
        session_description     = config['session_info']['session_description'],
        identifier              = config['metadata']['identifier'],
        session_start_time      = session_start_time,
        file_create_date        = config['metadata']['file_create_date'],
        experimenter            = config['general']['lab_info']['experimenter'],
        experiment_description  = config['general']['experiment_info']['experiment_description'],
        session_id              = config['session_info']['session_id'],
        lab                     = config['general']['lab_info']['lab'],                     
        institution             = config['general']['lab_info']['institution'],                                    
        keywords                = config['general']['experiment_info']['keywords'],
        protocol                = config['general']['experiment_info']['protocol'],
        related_publications    = config['general']['experiment_info']['related_publications'],
        surgery                 = config['general']['experiment_info']['surgery']
    )

    ################ CREATE SUBJECT ################################################
    ################################################################################
    nwbfile.subject = Subject(
        subject_id  = config['subject']['subject_id'],
        date_of_birth= config['subject']['date_of_birth'],
        species     = config['subject']['species'],
        sex         = config['subject']['sex'],
    )

    ################ CREATE HARDWARE LINKS #########################################
    ################################################################################
    nwbfile.create_device(
        name        = config['hardware']['system_name'], 
        description = config['hardware']['system_description'], 
        manufacturer= config['hardware']['system_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['adapter_manuf'], 
        description = config['hardware']['adapter_description'], 
        manufacturer= config['hardware']['adapter_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['monitor_name'], 
        description = config['hardware']['monitor_description'], 
        manufacturer= config['hardware']['monitor_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['photodiode_name'], 
        description = config['hardware']['photodiode_description'], 
        manufacturer= config['hardware']['photodiode_manuf']
    )
    
    nwbfile.create_device(
        name        = 'Software Used', 
        description = str(['Mworks Client: '+config['software']['mwclient_version'],\
                        'Mworks Server: '+config['software']['mwserver_version'],\
                        'OS: '+config['software']['OS'],\
                        'Intan :'+config['software']['intan_version']])
    )

    ################ CREATE ELECTRODE LINKS ########################################
    ################################################################################
    electrodes = nwbfile.create_device(
        name        = config['hardware']['electrode_name'], 
        description = config['hardware']['electrode_description'], 
        manufacturer= config['hardware']['electrode_manuf']
    )

    all_files = sorted(os.listdir(os.path.join(path, 'SpikeTimes')))
    
    name_accumulator = []
    for file in all_files:
        name_accumulator.append(read_names(file))
    names = np.vstack(name_accumulator)

    nwbfile.add_electrode_column(name="label", description="label of electrode")
    groups, count_groups = np.unique(names[:,0], return_counts =True)
    ids                  = names[:,1]
    counter              = 0
    # create ElectrodeGroups A, B, C, ..
    for group, count_group in zip(groups, count_groups):
        electrode_group = nwbfile.create_electrode_group(
            name        = "group_{}".format(group),
            description = "Serialnumber: {}. Adapter Version: {}".format(config['array_info']['array_{}'.format(group)]['serialnumber'],\
                            config['array_info']['array_{}'.format(group)]['adapterversion']),
            device      = electrodes,
            location    = 'hemisphere, region, subregion: '+str([config['array_info']['array_{}'.format(group)]['hemisphere'],\
                                config['array_info']['array_{}'.format(group)]['region'],
                                config['array_info']['array_{}'.format(group)]['subregion']]),
            position    = config['array_info']['array_{}'.format(group)]['position']
        )

        # create Electrodes 001, 002, ..., 032 in ElectrodeGroups per channel
        for ichannel in range(count_group):
            nwbfile.add_electrode(
                group       = electrode_group,
                label       = ids[counter],
                location    = 'row, col, elec'+str(json.loads(config['array_info']['intan_electrode_labeling_[row,col,id]'])[counter])
            )
            counter += 1     

    ################ ADD SPIKE TIMES ###############################################
    ################################################################################

    nwbfile.add_unit_column(name="unit", description="millisecond") 
    for filename, i in zip(sorted(os.listdir(os.path.join(path, 'SpikeTimes'))), range(len(os.listdir(os.path.join(path, 'SpikeTimes'))))):
        [assignment, number] = read_names(filename)
        file_path = os.path.join(path, 'SpikeTimes', filename)
        data = scipy.io.loadmat(file_path, squeeze_me=True,
                        variable_names='spike_time_ms')['spike_time_ms']
        nwbfile.add_unit(
            spike_times = data, 
            electrodes  = [i],
            electrode_group = nwbfile.electrode_groups[f'group_{assignment}'], 
            unit = 'ms'
        )

    ################ ADD TRIAL TIMES ###############################################
    ################################################################################
    last_spike = data[-1]
    del data
    with open(os.path.join(path, 'NWBInfo.txt')) as f:
        lines = f.readlines()
        line1 = lines[0].split(',')[0]
        StimOnnOff = [float(line1.split('/')[0]),float(line1.split('/')[1])] 

    on_start  = 0
    on_dur    = StimOnnOff[1]
    off_dur   = StimOnnOff[1]

    
    nwbfile.add_trial_column(name="unit", description="millisecond")
    while on_start < last_spike:

        nwbfile.add_trial(
            start_time= float(on_start),
            stop_time = float(on_start+on_dur),
            unit = 'ms')
    
        on_start += on_dur+off_dur

    return nwbfile

def get_psth_from_nwb(nwbfile, path, start_time, stop_time, timebin, n_stimuli=None):

    # Find the MWORKS File
    with open(os.path.join(path, 'NWBInfo.txt')) as f:
        lines = f.readlines()
        line2 = lines[0].split(',')[1]
        ind = line2.split('/').index('intanproc')
        mwk_file = glob.glob(os.path.join('/', *line2.split('/')[0:ind], 'mworksproc', \
                '_'.join(map(str, line2.split('/')[ind+1].split('_')[0:3]))+'*_mwk.csv'))   
        
    assert len(mwk_file) == 1
    mwk_file = mwk_file[0]
    assert os.path.isfile(mwk_file)==True

    ################ MODIFIED FROM THE SPIKE-TOOLS-CHONG CODE ######################
    ################################################################################
    
    mwk_data = pd.read_csv(mwk_file)
    mwk_data = mwk_data[mwk_data.fixation_correct == 1]
    if 'photodiode_on_us' in mwk_data.keys():
        samp_on_ms = np.asarray(mwk_data['photodiode_on_us']) / 1000.
        logging.info('Using photodiode signal for sample on time')
    else:
        samp_on_ms = np.asarray(mwk_data['samp_on_us']) / 1000.
        logging.info('Using MWorks digital signal for sample on time')
    
    # Load spikeTime file for current channel
    spikeTimes = nwbfile.units[:].spike_times
    # Re-order the psth to image x reps
    max_number_of_reps = max(np.bincount(mwk_data['stimulus_presented']))  # Max reps obtained for any image
    if max_number_of_reps == 0:
        exit()
    mwk_data['stimulus_presented'] = mwk_data['stimulus_presented'].astype(int)  # To avoid indexing errors
    
    if n_stimuli is None:
            image_numbers = np.unique(mwk_data['stimulus_presented'])  # TODO: if not all images are shown (for eg, exp cut short), you'll have to manually type in total # images
    else:
        image_numbers = np.arange(1,n_stimuli+1) # all of my image starts with #1

    timebase = np.arange(start_time, stop_time, timebin)
    PSTH = np.full((len(image_numbers), max_number_of_reps, len(timebase),spikeTimes.shape[0]), np.nan)

    for num in range(spikeTimes.shape[0]):
        spikeTime = np.asanyarray(spikeTimes[num])
        osamp = 10
        psth_bin = np.zeros((len(samp_on_ms), osamp*(stop_time-start_time)))
        psth_matrix = np.full((len(samp_on_ms), len(timebase)), np.nan)

        for i in range(len(samp_on_ms)):

            sidx = np.floor(osamp*(spikeTime[(spikeTime>=(samp_on_ms[i]+start_time))*(spikeTime<(samp_on_ms[i]+stop_time))]-(samp_on_ms[i]+start_time))).astype(int)
            psth_bin[i, sidx] = 1
            psth_matrix[i] = np.sum(np.reshape(psth_bin[i], [len(timebase), osamp*timebin]), axis=1)
        
        
        psth = np.full((len(image_numbers), max_number_of_reps, len(timebase)), np.nan)  # Re-ordered PSTH

        for i, image_num in enumerate(image_numbers):
            index_in_table = np.where(mwk_data.stimulus_presented == image_num)[0]
            selected_cells = psth_matrix[index_in_table, :]
            psth[i, :selected_cells.shape[0], :] = selected_cells

        logging.info(psth.shape)
        # Save psth data
        PSTH[:,:,:,num] = psth
        
    meta = {'start_time_ms': start_time, 'stop_time_ms': stop_time, 'tb_ms': timebin}
    cmbined_psth = {'psth': PSTH, 'meta': meta}

    return cmbined_psth
   


In [ ]:
inventory   = '/braintree/home/aliya277/dandi_brainscore/inventory'
start_time  = 0
stop_time   = 300
timebin     = 10
counter = 0


for folder, num_file in tqdm(zip(os.listdir(inventory), range(len(os.listdir(inventory)))), \
    total = len(os.listdir(inventory)), desc='Processing ...'):


    path = os.path.join(inventory, folder)
    SessionName = folder
    config_path = '/om/user/aliya277/dandi_brainscore'
    with open(os.path.join(config_path,"config_nwb.yaml") , "r") as f:
            config = yaml.load(f, Loader = yaml.FullLoader)

    print('Creating NWB file...')
    nwbfile = create_nwb(config,path)
    

    if 'h5Files' in os.listdir(path):
        print('Opening PSTH...')
        filename = os.listdir(os.path.join(path, 'h5Files'))[0]
        file = h5py.File(os.path.join(path, 'h5Files', filename),'r+') 
        data = file['psth'][:]
        file.close()


        print('Adding PSTH...')
        nwbfile.add_scratch(
            data,
            name="psth",
            description="psth, uncorrected [channels x stimuli x reps x timebins]",
            )
        
    else: counter +=1
    
    try: io.close()
    except:pass

    if os.path.isfile(os.path.join(path, f"{SessionName}.nwb")):
         os.remove(os.path.join(path, f"{SessionName}.nwb"))

    print('Saving NWB File..')
    io = NWBHDF5IO(os.path.join(path, f"{SessionName}.nwb"), "w") 
    io.write(nwbfile)
    io.close()
    print("File saved.")

        # psth    = get_psth_from_nwb(nwbfile, path, start_time, stop_time, timebin)
        # data    = psth['psth']

print(f'{counter} SpikeTimes do not have an h5 file.')


# extra

In [ ]:
def read_names(filename):
    assignment  = filename.split('.')[0].split('-')[1]
    number      = filename.split('.')[0].split('-')[2]
    return np.asarray([assignment, number])

def create_nwb(config, path):

    desired_timezone = pytz.timezone('US/Eastern')

    ################ CREATE NWB FILE WITH METADATA ################################
    ###############################################################################
    nwbfile = NWBFile(
        session_description     = config['session_info']['session_description'],
        identifier              = config['metadata']['identifier'],
        session_start_time      = desired_timezone.localize(config['metadata']['session_start_time']),
        file_create_date        = desired_timezone.localize(config['metadata']['file_create_date']),
        experimenter            = config['general']['lab_info']['experimenter'],
        experiment_description  = config['general']['experiment_info']['experiment_description'],
        session_id              = config['session_info']['session_id'],
        lab                     = config['general']['lab_info']['lab'],                     
        institution             = config['general']['lab_info']['institution'],                                    
        keywords                = config['general']['experiment_info']['keywords'],
        surgery                 = config['general']['experiment_info']['surgery']
    )

    ################ CREATE SUBJECT ################################################
    ################################################################################
    nwbfile.subject = Subject(
        subject_id  = config['subject']['subject_id'],
        date_of_birth= config['subject']['date_of_birth'],
        species     = config['subject']['species'],
        sex         = config['subject']['sex'],
    )

    ################ CREATE HARDWARE LINKS #########################################
    ################################################################################
    nwbfile.create_device(
        name        = config['hardware']['system_name'], 
        description = config['hardware']['system_description'], 
        manufacturer= config['hardware']['system_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['adapter_manuf'], 
        description = config['hardware']['adapter_description'], 
        manufacturer= config['hardware']['adapter_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['monitor_name'], 
        description = config['hardware']['monitor_description'], 
        manufacturer= config['hardware']['monitor_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['photodiode_name'], 
        description = config['hardware']['photodiode_description'], 
        manufacturer= config['hardware']['photodiode_manuf']
    )
    
    nwbfile.create_device(
        name        = 'Software Used', 
        description = str(['Mworks Client: '+config['software']['mwclient_version'],\
                        'Mworks Server: '+config['software']['mwserver_version'],\
                        'OS: '+config['software']['OS'],\
                        'Intan :'+config['software']['intan_version']])
    )

    ################ CREATE ELECTRODE LINKS ########################################
    ################################################################################
    electrodes = nwbfile.create_device(
        name        = config['hardware']['electrode_name'], 
        description = config['hardware']['electrode_description'], 
        manufacturer= config['hardware']['electrode_manuf']
    )

    all_files = sorted(os.listdir(os.path.join(path, 'SpikeTimes')))
    
    name_accumulator = []
    for file in all_files:
        name_accumulator.append(read_names(file))
    names = np.vstack(name_accumulator)

    nwbfile.add_electrode_column(name="label", description="label of electrode")
    groups, count_groups = np.unique(names[:,0], return_counts =True)
    ids                  = names[:,1]
    counter              = 0
    # create ElectrodeGroups A, B, C, ..
    for group, count_group in zip(groups, count_groups):
        if len(groups) == 6:
            electrode_description = "Serialnumber: {}. Adapter Version: {}".format(config['array_info']['array_{}'.format(group)]['serialnumber'],\
                            config['array_info']['array_{}'.format(group)]['adapterversion']),
        else: 
            electrode_description = "Serialnumber: {}".format(config['array_info']['array_{}'.format(group)]['serialnumber']),
                
        
        electrode_group = nwbfile.create_electrode_group(
            name        = "group_{}".format(group),
            description = electrode_description[0],
            device      = electrodes,
            location    = 'hemisphere, region, subregion: '+str([config['array_info']['array_{}'.format(group)]['hemisphere'],\
                                config['array_info']['array_{}'.format(group)]['region'],
                                config['array_info']['array_{}'.format(group)]['subregion']]),
            position    = config['array_info']['array_{}'.format(group)]['position']
        )

        # create Electrodes 001, 002, ..., 032 in ElectrodeGroups per channel
        for ichannel in range(count_group):
            nwbfile.add_electrode(
                group       = electrode_group,
                label       = ids[counter],
                location    = 'row, col, elec'+str(json.loads(config['array_info']['intan_electrode_labeling_[row,col,id]'])[counter])
            )
            counter += 1     

    ################ ADD SPIKE TIMES ###############################################
    ################################################################################

    nwbfile.add_unit_column(name="unit", description="millisecond") 
    for filename, i in zip(sorted(os.listdir(os.path.join(path, 'SpikeTimes'))), range(len(os.listdir(os.path.join(path, 'SpikeTimes'))))):
        [assignment, number] = read_names(filename)
        file_path = os.path.join(path, 'SpikeTimes', filename)
        data = scipy.io.loadmat(file_path, squeeze_me=True,
                        variable_names='spike_time_ms')['spike_time_ms']
        nwbfile.add_unit(
            spike_times = data, 
            electrodes  = [i],
            electrode_group = nwbfile.electrode_groups[f'group_{assignment}'], 
            unit = 'ms'
        )

    ################ ADD TRIAL TIMES ###############################################
    ################################################################################
    last_spike = data[-1]
    del data
    
    try: 
        [on, off] = config['session_info']['session_description'].split(', ')[3].split(': ')[-1].split("/")
        on_start  = 0
        on_dur    = int(on)
        off_dur   = int(off)

        
        nwbfile.add_trial_column(name="unit", description="millisecond")
        while on_start < last_spike:

            nwbfile.add_trial(
                start_time= float(on_start),
                stop_time = float(on_start+on_dur),
                unit = 'ms')
        
            on_start += on_dur+off_dur
    except: pass 

    ################ ADD PSTH IF AVAIL #############################################
    ################################################################################
    if 'h5Files' in os.listdir(path):

        filename = os.listdir(os.path.join(path, 'h5Files'))[0]
        file = h5py.File(os.path.join(path, 'h5Files', filename),'r+') 
        data = file['psth'][:]
        file.close()

        nwbfile.add_scratch(
            data,
            name="psth",
            description="psth, uncorrected [stimuli x reps x timebins x channels]",
            )
        

    return nwbfile


This file did not work: norm_FOSS.sub_pico.20230823_124104.proc
tail: output.log: Datei abgeschnitten
This file did not work: norm_FOSS.sub_pico.20230803_105856.proc
tail: output.log: Datei abgeschnitten
This file did not work: exp_gratingsAdap_s3.sub_pico.20230801_163355.proc
tail: output.log: Datei abgeschnitten
tail: output.log: Datei abgeschnitten
This file did not work: norm_FOSS.sub_pico.20230628_124854.proc
tail: output.log: Datei abgeschnitten
node069
tail: output.log: Datei abgeschnitten
tail: output.log: Datei abgeschnitten
node069
This file did not work: norm_FOSS.sub_pico.20220929_170635.proc
5127.proc
tail: output.log: Datei abgeschnitten
tail: output.log: Datei abgeschnitten
Saving NWB File..
File saved.
OSS.sub_pico.20230125_144402.proc
tail: output.log: Datei abgeschnitten
Saving NWB File..
File saved.
This file did not work: norm_FOSS.sub_pico.20230127_160227.proc
This file did not work: norm_FOSS.sub_pico.20230126_150653.proc
This file did not work: norm_FOSS.sub_pico.20230505_130540.proc
This file did not work: norm_FOSS.sub_pico.20230531_134209.proc
This file did not work: norm_FOSS.sub_pico.20230516_120137.proc
This file did not work: exp_robustness_guy_d1_v34.sub_pico.20230906_122650.proc
Saving NWB File..
File saved.
Saving NWB File..
File saved.
This file did not work: norm_FOSS.sub_pico.20230510_111142.proc
This file did not work: norm_FOSS.sub_pico.20230428_111937.proc
This file did not work: norm_FOSS.sub_pico.20220902_145351.proc
This file did not work: norm_FOSS.sub_pico.20220615_113442.proc
This file did not work: norm_FOSS.sub_pico.20220907_142157.proc
Saving NWB File..
File saved.
Saving NWB File..
File saved.
This file did not work: norm_FOSS.sub_pico.20230713_141950.proc
Saving NWB File..
File saved.
This file did not work: norm_FOSS.sub_pico.20230817_141050.proc
Saving NWB File..
File saved.
This file did not work: norm_FOSS.sub_pico.20230328_145456.proc
This file did not work: exp_gratingsAdap_s1.sub_pico.20230801_151117.proc
This file did not work: norm_FOSS.sub_pico.20230606_134244.proc
Saving NWB File..
File saved.
This file did not work: norm_FOSS.sub_pico.20230621_140747.proc
This file did not work: norm_FOSS.sub_pico.20230814_122811.proc
This file did not work: norm_FOSS.sub_pico.20230821_120107.proc
This file did not work: exp_Mayo_day_5.sub_pico.20230818_101245.proc
This file did not work: norm_HVM.sub_pico.20230404_142414.proc
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
This file did not work: norm_FOSS.sub_pico.20230626_131126.proc
This file did not work: norm_FOSS.sub_pico.20230629_132813.proc
This file did not work: norm_FOSS.sub_pico.20230711_142158.proc
This file did not work: norm_FOSS.sub_pico.20230630_135832.proc
This file did not work: norm_FOSS.sub_pico.20230725_113504.proc
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
Saving NWB File..
File saved.
This file did not work: norm_FOSS.sub_pico.20230830_110931.proc

In [5]:
start_time = 0
stop_time = 300
timebin = 10


def get_psth_from_nwb(nwbfile, path, start_time, stop_time, timebin, n_stimuli=None):

    # Find the MWORKS File
    with open(os.path.join(path, 'NWBInfo.txt')) as f:
        lines = f.readlines()
        line2 = lines[0].split(',')[1]
        ind = line2.split('/').index('intanproc')
        mwk_file = os.path.join('/', *line2.split('/')[0:ind], 'mworksproc', line2.split('/')[ind+1]+'_mwk.csv')
    assert os.path.isfile(mwk_file)==True

    ################ MODIFIED FROM THE SPIKE-TOOLS-CHONG CODE ######################
    ################################################################################
    
    mwk_data = pd.read_csv(mwk_file)
    mwk_data = mwk_data[mwk_data.fixation_correct == 1]
    if 'photodiode_on_us' in mwk_data.keys():
        samp_on_ms = np.asarray(mwk_data['photodiode_on_us']) / 1000.
        logging.info('Using photodiode signal for sample on time')
    else:
        samp_on_ms = np.asarray(mwk_data['samp_on_us']) / 1000.
        logging.info('Using MWorks digital signal for sample on time')
    
    # Load spikeTime file for current channel
    spikeTimes = nwbfile.units[:].spike_times
    # Re-order the psth to image x reps
    max_number_of_reps = max(np.bincount(mwk_data['stimulus_presented']))  # Max reps obtained for any image
    if max_number_of_reps == 0:
        exit()
    mwk_data['stimulus_presented'] = mwk_data['stimulus_presented'].astype(int)  # To avoid indexing errors
    
    if n_stimuli is None:
            image_numbers = np.unique(mwk_data['stimulus_presented'])  # TODO: if not all images are shown (for eg, exp cut short), you'll have to manually type in total # images
    else:
        image_numbers = np.arange(1,n_stimuli+1) # all of my image starts with #1

    timebase = np.arange(start_time, stop_time, timebin)
    PSTH = np.full((len(image_numbers), max_number_of_reps, len(timebase),spikeTimes.shape[0]), np.nan)

    for num in range(spikeTimes.shape[0]):
        spikeTime = np.asanyarray(spikeTimes[num])
        osamp = 10
        psth_bin = np.zeros((len(samp_on_ms), osamp*(stop_time-start_time)))
        psth_matrix = np.full((len(samp_on_ms), len(timebase)), np.nan)

        for i in range(len(samp_on_ms)):

            sidx = np.floor(osamp*(spikeTime[(spikeTime>=(samp_on_ms[i]+start_time))*(spikeTime<(samp_on_ms[i]+stop_time))]-(samp_on_ms[i]+start_time))).astype(int)
            psth_bin[i, sidx] = 1
            psth_matrix[i] = np.sum(np.reshape(psth_bin[i], [len(timebase), osamp*timebin]), axis=1)
        
        
        psth = np.full((len(image_numbers), max_number_of_reps, len(timebase)), np.nan)  # Re-ordered PSTH

        for i, image_num in enumerate(image_numbers):
            index_in_table = np.where(mwk_data.stimulus_presented == image_num)[0]
            selected_cells = psth_matrix[index_in_table, :]
            psth[i, :selected_cells.shape[0], :] = selected_cells

        logging.info(psth.shape)
        # Save psth data
        PSTH[:,:,:,num] = psth
        
    meta = {'start_time_ms': start_time, 'stop_time_ms': stop_time, 'tb_ms': timebin}
    cmbined_psth = {'psth': PSTH, 'meta': meta}

    return cmbined_psth
    
# psth = get_psth_from_nwb(nwbfile, path, start_time, stop_time, timebin)


In [ ]:
def save_nwb(nwbfile, path):
    
    SessionName = path.split('/')[-1]

    # Crete Temporary path on BrainTree (Openmind Did not Allow Saving of H5 Files.)
    temp_path = '/braintree/home/aliya277/dandi_brainscore/inventory'
    try: os.mkdir(temp_path)
    except: pass

    # Save NWB File on Braintree
    io = NWBHDF5IO(os.path.join(temp_path, f"{SessionName}.nwb"), "w") 
    io.write(nwbfile)
    io.close()

    # Copy NWB to Inventory Directory on Openmind
    shutil.copy2(os.path.join(temp_path, f"{SessionName}.nwb"), os.path.join(path, f"{SessionName}.nwb"))   
    
    # Remove Temporary File and Folder on Braintree
    os.remove(os.path.join(temp_path, f"{SessionName}.nwb"))
    os.rmdir(os.path.join(temp_path))

    return nwbfile

In [65]:
data = psth['psth']

nwbfile.add_scratch(
    data,
    name="psth",
    description="psth, uncorrected [channels x stimuli x reps x timebins]",
)

ValueError: 'psth' already exists in NWBFile 'root'

In [64]:
nwbfile.scratch['psth'].shape

(192, 86, 21, 30)

In [56]:
h5path = '/om/user/aliya277/inventory/norm_FOSS.sub_pico.20230823_124104.proc/h5Files/230823.pico.rsvp.normalizers.experiment_psth_raw.h5'

def open_h5(path):
    
    filename = path.split('/')[-1]
    # Crete Temporary path on BrainTree (Openmind Did not Allow Saving of H5 Files.)
    temp_path = '/braintree/home/aliya277/dandi_brainscore/inventory'
    try: os.mkdir(temp_path)
    except: pass


    # Copy File to Temporary Path
    shutil.copy2(os.path.join(path), os.path.join(temp_path, filename))   

    file = h5py.File(os.path.join(temp_path, filename),'r+')    

    return file

def close_h5(path):

    filename = path.split('/')[-1]
    temp_path = '/braintree/home/aliya277/dandi_brainscore/inventory'
    os.remove(os.path.join(temp_path, filename))
    os.rmdir(temp_path)

file = open_h5(h5path)


In [71]:
np.array_equal(psth['psth'], file['psth'][:], equal_nan=True)

True

In [ ]:
nwbfile.units[:].spike_times


id
0      [69.2, 135.55, 207.55, 242.9, 281.95, 331.5, 3...
1      [22.1, 57.3, 63.9, 135.35, 167.1, 175.6, 225.8...
2      [58.1, 69.15, 167.7, 171.1, 171.3, 286.9, 362....
3      [16.9, 158.04999999999998, 237.9, 304.09999999...
4      [49.75, 68.9, 87.65, 115.1, 342.95, 412.650000...
                             ...                        
187    [2.6, 5.3, 81.3, 155.35, 185.35, 251.649999999...
188    [35.2, 164.14999999999998, 164.70000000000002,...
189    [35.2, 155.35, 251.8, 286.95, 287.1, 287.4, 29...
190    [2.6, 162.35, 198.8, 251.5, 251.7, 287.2, 294....
191    [46.449999999999996, 73.85, 81.44999999999999,...
Name: spike_times, Length: 192, dtype: object

In [5]:
nwb2widget(nwbfile)